In [2]:
!pip install lifelines
import pandas as pd
import numpy as np
import os
import statsmodels.api as sm
from sklearn.linear_model import LogisticRegression
from lifelines.statistics import logrank_test
from lifelines import CoxPHFitter
import warnings
warnings.filterwarnings("ignore")
from statsmodels.stats.multitest import multipletests
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from sklearn.metrics import accuracy_score, classification_report
from sklearn import svm
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
full_data = pd.read_csv('Full Data.txt', index_col = [0])

In [4]:
full_data

,gender,ajcc_pathologic_tumor_stage,tumor_status,DSS,DSS.time,TP53,TTN,FAT1,MUC16,CSMD3,...,NIPBL,CEACAM5,CEACAM6,CLCA4,EGFR,Missense_Mutation,Nonsense_Mutation,Nonstop_Mutation,Splice_Site,age
patient_id,,,,,,,,,,,,,,,,,,,,,
TCGA-4P-AA8J,1,0,0,0.0,0.279452,822.678,1157.3300,1620.71,1004.4700,1.2325,...,1595.440,56.0776,423.3550,502.2340,722.847,92,8,0,2,1
TCGA-BA-4074,1,0,1,1.0,1.265753,1805.510,103.6870,6415.45,17.3422,0.3854,...,1605.890,153.3820,275.9330,12.7176,1267.900,106,8,0,1,1
TCGA-BA-4075,1,1,1,1.0,0.775342,383.760,295.9180,4162.40,1.0720,36.9824,...,907.410,18.2232,45.5581,10.1836,4833.430,113,10,0,4,0
TCGA-BA-4077,0,0,1,1.0,3.106849,1929.260,207.3870,7246.87,432.6270,0.0000,...,1174.690,2392.6900,2437.9200,720.8850,2371.940,234,20,0,6,0
TCGA-BA-5149,1,0,1,1.0,2.208219,165.298,347.9540,8848.73,110.5410,0.3422,...,1798.770,37.3032,447.6390,5.4757,6934.280,96,6,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
TCGA-UF-A7JO,0,0,1,1.0,1.728767,1604.880,176.1930,1881.36,347.7040,2.8008,...,854.256,1498.8500,2014.6000,82.8248,530.911,135,9,0,4,1
TCGA-UF-A7JS,1,0,1,1.0,1.863014,2457.400,1379.4300,1339.31,243.6470,19.4320,...,982.063,358.7440,294.4690,32.8849,246.637,50,1,0,1,1
TCGA-UF-A7JT,0,0,1,1.0,2.720548,1684.850,149.5190,3430.74,213.8530,14.2857,...,1009.520,49.7835,209.5240,200.4330,790.905,152,10,0,3,1


# Log-rank test

In [5]:
def quantile_gene(df, x):
    q1 = df[x].quantile(0.1)
    q3 = df[x].quantile(0.9)
    df[x + 'binary'] = df[x].apply(lambda x: 0 if x <= q1 else 1 if x >= q3 else np.nan)

In [6]:
def logrank(df, lis, time):
    dic ={}
    data = df[df['DSS.time'] <= time]
    for i in lis:
        quantile_gene(data, i)
        low_expression = data[data[i + 'binary'] == 0]
        high_expression =  data[data[i +'binary'] == 1]
        # logrank
        results = logrank_test(high_expression['DSS.time'], low_expression['DSS.time'],
                       high_expression['DSS'], low_expression['DSS'])
        z_value = results.test_statistic
        if z_value > 1.96 or z_value < -1.96:
            dic[i] = z_value
    
    # return a dictionary of name and z value
    return dic

In [ ]:
gene_expression = list(full_data.drop(columns = ['gender', 'ajcc_pathologic_tumor_stage', 'tumor_status', 'DSS' ,
                                      'DSS.time', 'Missense_Mutation', 'Nonsense_Mutation', 
                                      'Nonstop_Mutation', 'Splice_Site', 'age']).columns)

In [ ]:
# for gene expression
a = logrank(full_data,gene_expression, 2) # example: use full data and 2 years as limit

In [ ]:
time_limit = [1, 2, 5, 7, 10]
feature_dic = {}
for i in time_limit:
    dic = logrank(full_data, gene_expression, i)
    feature_dic[i] = list(dic.keys())

In [ ]:
#select top n genes by sort z values as features from full_data

In [7]:
mutation = ['Missense_Mutation', 'Nonsense_Mutation', 'Nonstop_Mutation', 'Splice_Site']

In [9]:
# for mutation
logrank(full_data,mutation, 2) # example: use full data and 2 years as limit

{'Missense_Mutation': 2.2088500784074974,
 'Nonsense_Mutation': 1.9839793008189441,
 'Splice_Site': 5.71020812335661}

In [10]:
time_limit = range(1,11)
feature_dic = {}
for i in time_limit:
    dic = logrank(full_data, mutation, i)
    feature_dic[i] = list(dic.keys())

# Classification

In [ ]:
#select top n mutations by z values as features columns from full_data

In [13]:
def classification_svm(df, features):
    # get features you select
    X = df[features]
    y = df['DSS']

    # Split the dataset into training and testing datasets
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
    # SVM
   
    # Define the model
    svm_clf = svm.SVC()

    # Train the model
    svm_clf.fit(X_train, y_train)
   
    # Predict on the test set
    svm_predictions = svm_clf.predict(X_test)

    # Evaluate the model
    return accuracy_score(y_test, svm_predictions)
    #print("SVM Accuracy:", accuracy_score(y_test, svm_predictions))

In [14]:
acc_svm = []
for i in time_limit:
    lis = []
    for k in range(100):
        accuracy = classification_svm(full_data, feature_dic[i])
        lis.append(accuracy)
    mean_acc = sum(lis)/len(lis)
    acc_svm.append(mean_acc)
    print('For time limit ' + str(i) + ' and mutation features from log rank test')
    print('SVM Accuracy ' + str(mean_acc))

For time limit 1 and mutation features from log rank test
SVM Accuracy 0.7037777777777778
For time limit 2 and mutation features from log rank test
SVM Accuracy 0.7076666666666667
For time limit 3 and mutation features from log rank test
SVM Accuracy 0.7041111111111111
For time limit 4 and mutation features from log rank test
SVM Accuracy 0.7035555555555555
For time limit 5 and mutation features from log rank test
SVM Accuracy 0.7036666666666664
For time limit 6 and mutation features from log rank test
SVM Accuracy 0.7027777777777777
For time limit 7 and mutation features from log rank test
SVM Accuracy 0.7095555555555557
For time limit 8 and mutation features from log rank test
SVM Accuracy 0.7149999999999997
For time limit 9 and mutation features from log rank test
SVM Accuracy 0.7046666666666664
For time limit 10 and mutation features from log rank test
SVM Accuracy 0.7070000000000001


In [30]:
def classification_logistic(df, features):
    # get features you select
    X = df[features]
    y = df['DSS']

    # Split the dataset into training and testing datasets
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
    # add classification
    # choose any classfication to do prediction
    # Define the model
    log_reg = LogisticRegression()

    # Train the model
    log_reg.fit(X_train, y_train)
    log_reg_predictions = log_reg.predict(X_test)

    # Evaluate the model
    return accuracy_score(y_test, log_reg_predictions)
    #print("Logistic Regression Accuracy:", accuracy_score(y_test, log_reg_predictions))

In [31]:
for i in time_limit:
    lis = []
    for k in range(10000):
        accuracy = classification_logistic(full_data, feature_dic[i])
        lis.append(accuracy)
    mean_acc = sum(lis)/len(lis)
    print('For time limit ' + str(i) + ' and mutation features from log rank test')
    print('Logistic Regression Accuracy ' + str(mean_acc))

For time limit 1 and mutation features from log rank test
Logistic Regression Accuracy 0.7046833333333243
For time limit 2 and mutation features from log rank test
Logistic Regression Accuracy 0.7037922222222153
For time limit 3 and mutation features from log rank test
Logistic Regression Accuracy 0.7069488888888857
For time limit 4 and mutation features from log rank test
Logistic Regression Accuracy 0.7063299999999925
For time limit 5 and mutation features from log rank test
Logistic Regression Accuracy 0.7048244444444353
For time limit 6 and mutation features from log rank test
Logistic Regression Accuracy 0.7045133333333272
For time limit 7 and mutation features from log rank test
Logistic Regression Accuracy 0.7041977777777687
For time limit 8 and mutation features from log rank test
Logistic Regression Accuracy 0.7054088888888834
For time limit 9 and mutation features from log rank test
Logistic Regression Accuracy 0.7050855555555473
For time limit 10 and mutation features from l

In [23]:
def classification_randomforest(df, features):
    # get features you select
    X = df[features]
    y = df['DSS']

    # Split the dataset into training and testing datasets
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
    # add classification
    # choose any classfication to do prediction
    # Define the model
    rf_clf = RandomForestClassifier()

    # Train the model
    rf_clf.fit(X_train, y_train)
    # Predict on the test set
    rf_predictions = rf_clf.predict(X_test)

    # Evaluate the model
    return accuracy_score(y_test, rf_predictions)
    #print("Random Forest Accuracy:", accuracy_score(y_test, rf_predictions))

In [26]:
acc_svm = []
for i in time_limit:
    lis = []
    for k in range(100):
        accuracy = classification_randomforest(full_data, feature_dic[i])
        lis.append(accuracy)
    mean_acc = sum(lis)/len(lis)
    acc_svm.append(mean_acc)
    print('For time limit ' + str(i) + ' and mutation features from log rank test')
    print('Random Forest Accuracy ' + str(mean_acc))

For time limit 1 and mutation features from log rank test
Random Forest Accuracy 0.6770000000000002
For time limit 2 and mutation features from log rank test
Random Forest Accuracy 0.7300000000000003
For time limit 3 and mutation features from log rank test
Random Forest Accuracy 0.6973333333333338
For time limit 4 and mutation features from log rank test
Random Forest Accuracy 0.690888888888889
For time limit 5 and mutation features from log rank test
Random Forest Accuracy 0.6580000000000003
For time limit 6 and mutation features from log rank test
Random Forest Accuracy 0.6713333333333334
For time limit 7 and mutation features from log rank test
Random Forest Accuracy 0.6571111111111114
For time limit 8 and mutation features from log rank test
Random Forest Accuracy 0.6502222222222224
For time limit 9 and mutation features from log rank test
Random Forest Accuracy 0.649444444444445
For time limit 10 and mutation features from log rank test
Random Forest Accuracy 0.6476666666666668


In [20]:
def best_estimator(df, features):
    # get features you select
    X = df[features]
    y = df['DSS']

    # Split the dataset into training and testing datasets
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
    # add classification
    # choose any classfication to do prediction
    # Define the model
    rf_clf = RandomForestClassifier()
    param_grid = { 
    'n_estimators': [10, 50, 100, 200],
    'max_features': ['auto', 'sqrt', 'log2'],
    'max_depth' : [4,5,6,7,8],
    'criterion' :['gini', 'entropy']}
    # Train the model
    CV_rf = GridSearchCV(estimator=rf_clf, param_grid=param_grid, cv = 5)
    CV_rf.fit(X_train, y_train)
    # Predict on the test set
    return CV_rf.best_params_

In [21]:
# get the best parameter for each time based on each feature
para_dic = {}
for i in time_limit:
    para_dic[i] = best_estimator(full_data, feature_dic[i])

In [22]:
def classifcation_randomforestcv(df, features, best_para):
    # get features you select
    X = df[features]
    y = df['DSS']

    # Split the dataset into training and testing datasets
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
    # add classification
    # choose any classfication to do prediction
    # Define the model
    rf_best = RandomForestClassifier(n_estimators= best_para['n_estimators'], max_features=best_para['max_features'], max_depth=best_para['max_depth'], criterion=best_para['criterion'])
    rf_best.fit(X_train, y_train)

    predictions = rf_best.predict(X_test)
    accuracy = accuracy_score(y_test, predictions)
    cv_score = cross_val_score(rf_best, X, y, cv=5)
    # Evaluate the model
    return accuracy, cv_score.mean()

In [27]:
for i in time_limit:
    lis_grid = []
    lis_cv = []
    for k in range(10):
        grid_accuracy, cv_score = classifcation_randomforestcv(full_data, feature_dic[i],para_dic[i])
        lis_grid.append(grid_accuracy)
        lis_cv.append(cv_score)
    mean_grid = sum(lis_grid)/len(lis_grid)
    mean_cv =  sum(lis_cv)/len(lis_cv)
    print('For time limit ' + str(i) + ' and mutation features from log rank test')
    print('Random Forest GridSearchCV Accuracy ' + str(mean_grid))
    print('Random Forest cross validation Accuracy ' + str(mean_cv))

For time limit 1 and mutation features from log rank test
Random Forest GridSearchCV Accuracy 0.7055555555555556
Random Forest cross validation Accuracy 0.6922222222222222
For time limit 2 and mutation features from log rank test
Random Forest GridSearchCV Accuracy 0.7344444444444445
Random Forest cross validation Accuracy 0.6813333333333335
For time limit 3 and mutation features from log rank test
Random Forest GridSearchCV Accuracy 0.6933333333333334
Random Forest cross validation Accuracy 0.6964444444444445
For time limit 4 and mutation features from log rank test
Random Forest GridSearchCV Accuracy 0.7177777777777778
Random Forest cross validation Accuracy 0.6957777777777778
For time limit 5 and mutation features from log rank test
Random Forest GridSearchCV Accuracy 0.7288888888888889
Random Forest cross validation Accuracy 0.6857777777777778
For time limit 6 and mutation features from log rank test
Random Forest GridSearchCV Accuracy 0.7233333333333334
Random Forest cross validat